# Import Libraries

In [52]:
# DataFrame
import pandas as pd 


# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
# Utility
import warnings
warnings.filterwarnings('ignore')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import re
import string
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [2]:
# Construct a tf.data.Dataset
data = pd.read_csv(r"C:\Semester 4\PP\NLP_Sentimen\datasentimen\training.1600000.processed.noemoticon.csv",encoding='latin', names = ['polarity','id','date','query','user','text'])

In [3]:
data

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


## Download Required NLTK stopword, Lematizer

nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

In [4]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'of', 'am', 'under', 'until', 'so', 'into', 'further', 'from', 'she', 't', 'very', 'by', 'at', "you'd", 'themselves', 'where', 'but', 'more', "you're", 'those', 'own', 'on', 'then', 'needn', "shouldn't", 'because', "weren't", 'her', 'had', 'wasn', 'some', 'than', 'theirs', 'have', 'doing', 'for', 'against', 'after', "isn't", "she's", "you've", 'its', 'should', 'will', 'his', 'over', 'd', "aren't", 'was', 'itself', "don't", 'our', 'few', 'with', 'having', 'shan', 'were', 'both', 'them', "should've", 'that', 've', 'through', 'hers', 'no', 'we', 'while', 's', 'why', 'are', 'can', 'you', 'been', 'it', "won't", 'again', 'the', 'what', 'such', 'does', "hadn't", 'between', 'down', "couldn't", 'any', "it's", 'do', 'each', 'herself', 'here', 'whom', "that'll", 'ain', 'and', "didn't", 'not', 'off', 'your', 'weren', 'o', 'below', 'i', 'this', 're', 'has', 'me', 'these', 'about', 'hasn', 'as', 'my', 'how', 'wouldn', 'during', 'yourselves', 'up', 'their', "wasn't", 'when', 'same', 'he', 'shouldn',

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krisn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krisn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krisn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Cleaning

In [6]:

data['polarity'] = data['polarity'].replace(4,1)

positives = data['polarity'][data.polarity == 1 ]
negatives = data['polarity'][data.polarity == 0 ]

print('Total length of the data is:         {}'.format(data.shape[0]))
print('No. of positve tagged sentences is:  {}'.format(len(positives)))
print('No. of negative tagged sentences is: {}'.format(len(negatives)))

Total length of the data is:         1600000
No. of positve tagged sentences is:  800000
No. of negative tagged sentences is: 800000


In [7]:
def process_tweets(tweet):
    tweet = tweet.lower()

    # Remove URLs
    url_pattern = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
    tweet = re.sub(url_pattern, '', tweet)

    # Remove username
    username_pattern = r'@[a-zA-Z0-9_]+'
    tweet = re.sub(username_pattern, '', tweet)

    tweet = tweet.split()
    # Remove punctuations
    tokens_without_punctuation = [token for token in tweet if token not in string.punctuation]

    # Reconstruct the text without the username
    tweet = ' '.join(tokens_without_punctuation)

    # Tokenize words
    tokens = nltk.word_tokenize(tweet)

    # Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]

    # Reduce words to their word stems
    word_lemmatizer = WordNetLemmatizer()
    final_words = [word_lemmatizer.lemmatize(w) for w in final_tokens if len(w) > 1]

    return ' '.join(final_words)

In [8]:
data['processed_tweets'] = data['text'].apply(lambda x: process_tweets(x))
print('Text Preprocessing complete.')

Text Preprocessing complete.


# Feature Extraction and splitting


In [10]:
X = data['processed_tweets']
y = data['polarity']

In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

vectorizer = TfidfVectorizer(ngram_range=(1,3))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Modelling

In [49]:


init_models = {
    'Logistic Regression': LogisticRegression()
}

accuracy = []
recall = []
precision = []
f1 = []
models_names = []

for key, model in init_models.items():
    models_names.append(key)
    model.fit(X_train, y_train)  # Fit the model on training data
    y_pred = model.predict(X_test)  # Make predictions on test data
    accuracy.append(accuracy_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))

models_scores = pd.DataFrame({'Model Name': models_names, 'Accuracy': accuracy, 'Recall': recall, 'Precision': precision, 'F1': f1})
models_scores.head()


,Model Name,Accuracy,Recall,Precision,F1
0,Logistic Regression,0.793472,0.788493,0.796177,0.792316


In [12]:


init_models = {
   
    'LightGBM': lgb.LGBMClassifier()
}

accuracy = []
recall = []
precision = []
f1 = []
models_names = []

for key, model in init_models.items():
    models_names.append(key)
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)  
    accuracy.append(accuracy_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))

models_scores = pd.DataFrame({'Model Name': models_names, 'Accuracy': accuracy, 'Recall': recall, 'Precision': precision, 'F1': f1})
models_scores.head()


,Model Name,Accuracy,Recall,Precision,F1
0,LightGBM,0.741931,0.817952,0.709765,0.760028


In [14]:


init_models = {

    'Gaussian Naive Bayes': BernoulliNB()
 
}

accuracy = []
recall = []
precision = []
f1 = []
models_names = []

for key, model in init_models.items():
    models_names.append(key)
    model.fit(X_train, y_train)  # Convert X_train to dense format
    y_pred = model.predict(X_test) # Convert X_test to dense format
    accuracy.append(accuracy_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))

models_scores = pd.DataFrame({'Model Name': models_names, 'Accuracy': accuracy, 'Recall': recall, 'Precision': precision, 'F1': f1})
models_scores.head()


,Model Name,Accuracy,Recall,Precision,F1
0,Gaussian Naive Bayes,0.778066,0.826609,0.753232,0.788216


In [ ]:



init_models = {

    'LinearSVC': LinearSVC()
 
}

accuracy = []
recall = []
precision = []
f1 = []
models_names = []

for key, model in init_models.items():
    models_names.append(key)
    model.fit(X_train, y_train)  # Convert X_train to dense format
    y_pred = model.predict(X_test) # Convert X_test to dense format
    accuracy.append(accuracy_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))

models_scores = pd.DataFrame({'Model Name': models_names, 'Accuracy': accuracy, 'Recall': recall, 'Precision': precision, 'F1': f1})
models_scores.head()


,Model Name,Accuracy,Recall,Precision,F1
0,LinearSVC,0.792588,0.787323,0.795453,0.791367


In [46]:
texts = [
    "I absolutely loved the movie. It was fantastic!",
    "The restaurant had amazing food and excellent service.",
    "I was deeply disappointed by the book. It did not meet my expectations.",
    "The customer support was terrible. I had a horrible experience.",
    "The concert was electrifying. The performers were incredible.",
    "The product is of poor quality. I regret buying it.",
    "The vacation was a dream come true. It exceeded all my expectations.",
    "The company's policies are unfair and unethical.",
    "The new software update has greatly improved performance. I'm impressed.",
    "The service at the hotel was subpar. I wouldn't recommend it."
]
preds=[]
for i in texts :
    text=process_tweets(i)
    text=vectorizer.transform([text])
    preds.append(model.predict(text))

preds= [item[0] for item in preds]

In [47]:
pd.DataFrame({'text': texts,'sentimen prediction' : preds})

,text,sentimen prediction
0,I absolutely loved the movie. It was fantastic!,1
1,The restaurant had amazing food and excellent ...,1
2,I was deeply disappointed by the book. It did ...,0
3,The customer support was terrible. I had a hor...,0
4,The concert was electrifying. The performers w...,1
5,The product is of poor quality. I regret buyin...,0
6,The vacation was a dream come true. It exceede...,1
7,The company's policies are unfair and unethical.,0
8,The new software update has greatly improved p...,1
9,The service at the hotel was subpar. I wouldn'...,1
